# These are all of the imports we would require

In [1]:
import numpy as np
from functools import partial
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
import urllib.request
import os
import zipfile

# Download Google's Inception Network

In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip'
data_dir = './data/'
model_name = os.path.split(url)[-1]
local_zip_file = os.path.join(data_dir, model_name)
if not os.path.exists(local_zip_file):
    # Download
    model_url = urllib.request.urlopen(url)
    with open(local_zip_file, 'wb') as output:
        output.write(model_url.read())
    # Extract
    with zipfile.ZipFile(local_zip_file, 'r') as zip_ref:
        zip_ref.extractall(data_dir)
        img_noise = np.random.uniform(size=(224,224,3)) + 100.0
        
# start with a gray image with a little noise
img_noise = np.random.uniform(size=(224,224,3)) + 100.0
model_fn = 'tensorflow_inception_graph.pb'

 # Creating Tensorflow session and loading the model

In [3]:
 #Step 2 - Creating Tensorflow session and loading the model
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
with tf.gfile.FastGFile(os.path.join(data_dir, model_fn), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
t_input = tf.placeholder(np.float32, name='input') # define the input tensor
imagenet_mean = 117.0
t_preprocessed = tf.expand_dims(t_input-imagenet_mean, 0)
tf.import_graph_def(graph_def, {'input':t_preprocessed})
    
layers = [op.name for op in graph.get_operations() if op.type=='Conv2D' and 'import/' in op.name]
feature_nums = [int(graph.get_tensor_by_name(name+':0').get_shape()[-1]) for name in layers]
    
print('Number of layers', len(layers))
print('Total number of feature channels:', sum(feature_nums))

Number of layers 59
Total number of feature channels: 7548


<p>1. showarray()  is used to output the image
<br> 2. T() is used to get the output tensor of the specified layer
<br> 3. tffunc() converts the TF-graph to a regular one
<br> 4. resize() is used to resize the image for each iteration to allow for scaling
<br> 5. calc_grad_tiled() is used to blur the boundaries between each tile(octave) to allow for effective gradient ascent</p>

In [4]:
def showarray(a):
    a = np.uint8(np.clip(a, 0, 1)*255)
    plt.imshow(a)
    plt.show()    
    
def T(layer):
    '''Helper for getting layer output tensor'''
    return graph.get_tensor_by_name("import/%s:0"%layer)
    
def tffunc(*argtypes):
    '''Helper that transforms TF-graph generating function into a regular one.
    See "resize" function below.
    '''
    placeholders = list(map(tf.placeholder, argtypes))
    def wrap(f):
        out = f(*placeholders)
        def wrapper(*args, **kw):
            return out.eval(dict(zip(placeholders, args)), session=kw.get('session'))
        return wrapper
    return wrap
    
def resize(img, size):
    img = tf.expand_dims(img, 0)
    return tf.image.resize_bilinear(img, size)[0,:,:,:]
resize = tffunc(np.float32, np.int32)(resize)
    
def calc_grad_tiled(img, t_grad, tile_size=512):
    '''Compute the value of tensor t_grad over the image in a tiled way.
    Random shifts are applied to the image to blur tile boundaries over 
    multiple iterations.'''
    sz = tile_size
    h, w = img.shape[:2]
    sx, sy = np.random.randint(sz, size=2)
    img_shift = np.roll(np.roll(img, sx, 1), sy, 0)
    grad = np.zeros_like(img)
    for y in range(0, max(h-sz//2, sz),sz):
        for x in range(0, max(w-sz//2, sz),sz):
            sub = img_shift[y:y+sz,x:x+sz]
            g = sess.run(t_grad, {t_input:sub})
            grad[y:y+sz,x:x+sz] = g
    return np.roll(np.roll(grad, -sx, 1), -sy, 0)

# Deep Dream is function is declared here

In [5]:
def render_deepdream(t_obj, img0=img_noise,iter_n=10, step=1.5, octave_n=10, octave_scale=1.4):
    t_score = tf.reduce_mean(t_obj) # defining the optimization objective
    t_grad = tf.gradients(t_score, t_input)[0] # behold the power of automatic differentiation!
    
    # split the image into a number of octaves
    img = img0
    octaves = []
    for _ in range(octave_n-1):
        hw = img.shape[:2]
        lo = resize(img, np.int32(np.float32(hw)/octave_scale))
        hi = img-resize(lo, hw)
        img = lo
        octaves.append(hi)
        
    # generate details octave by octave
    for octave in range(octave_n):
        if octave>0:
            hi = octaves[-octave]
            img = resize(img, hi.shape[:2])+hi
        for _ in range(iter_n):
            g = calc_grad_tiled(img, t_grad)
            img += g*(step / (np.abs(g).mean()+1e-7))
            
        #this will usually be like 3 or 4 octaves
        #Step 5 output deep dream image via matplotlib
        showarray(img/255.0) 

# Picking a layer to enhance our image

In [6]:
#Step 3 - Pick a layer to enhance our image
layer = 'mixed4d_3x3_bottleneck_pre_relu'
channel = 139 # picking some feature channel to visualize

# Loading image and calling deepdream

In [1]:
#open image
img0 = PIL.Image.open('pilatus800.jpg')
img0 = np.float32(img0)
     
#Step 4 - Apply gradient ascent to that layer
render_deepdream(tf.square(T('mixed4c')), img0)

NameError: name 'PIL' is not defined